The following script will be used to adjust the already create models further with gene deletions to further increase the similarity to the *e. coli* pathways, that are described in Huang, Zhongshi, et al paper. 

In [1]:
# This is just importing of the model and some tools to work with the GSM.
import numpy as np
from cobra.io import read_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
import pandas as pd
# Read model (central metabolism model of Escherichia coli)
model = read_sbml_model("iML1515.xml")
%run -i create_pathway.ipynb #runs script, has 

Exception: File `'create_pathway.ipynb.py'` not found.

In [ ]:
# creating the models with the respective pathways
nam_model = create_pathway(model.copy(),1)
na_model = create_pathway(model.copy(),2)
nr_model = create_pathway(model.copy(),3)
models = [(nam_model,'NAMPT'),(na_model,'NMN Synthetase'),(nr_model,'DeoD')]

In [ ]:
import pandas
from time import time

from cobra.io import load_model
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

NAM Model: For the Nam Model it is not necessary to Knockout the purR gene, because it is not available in the GSM 

NA Model: For the NA Model it is necessary to knock out the pncC gene. The pncC is known as the gene: b2700 in the iML1515 model and is associated to the reaction NMNDA 

In [ ]:
print('complete model: ', na_model.optimize())
with na_model:
    na_model.genes.b2700.knock_out()
    print('pncC knocked out: ', na_model.optimize())
    

NR Model: For the NR Model it is imortant to knock out the nadR and ushA genes. The UshA gene is known as the b0480 gene in the iML1515 model and is associated to many different reactions. the nadR is known as the b4390 gene and associated for the NMNAT reaction.  

In [ ]:
print('complete model: ', nr_model.optimize())
with nr_model:
    nr_model.genes.b0480.knock_out()
    print('ushA knocked out: ', nr_model.optimize())
    nr_model.genes.b4390.knock_out()
    print('nadR knocked out: ', nr_model.optimize())

In [ ]:
import logging
from ast import And, BoolOp, Name, NodeTransformer
from functools import partial
from typing import TYPE_CHECKING, Iterable, List, Optional, Set, Tuple, Union

from cobra.util import get_context

In [ ]:
if TYPE_CHECKING:
    from cobra import Gene, Metabolite, Model, Reaction

In [ ]:
def remove_genes(
    model:"na_model",
    gene_list: Union[List["b2700"],Set["Gene"],List[str],Union[str]],
    remove_reactions: bool = True,) -> None:

In [ ]:
def remove_genes(
    model: "na_model"
    gene_list:Union[List["b2700"],Set["Gene"],List[str],Union[str]],
    remove_reaction:bool = True,)
-> None: 

double_gene_deletion(
    nr_model, nr_model.genes[-5:]).round(2)

In [31]:


def knock_out_model_genes(
    model: "Model",
    gene_list: Iterable[Union["b2700", int, str]],
) -> List["Reaction"]:
    """Disable the genes in `gene_list`.

    It sets the bounds to "zero" for reactions catalysed by the genes in
    `gene_list` if deleting the genes would stop the reactions from proceeding.

    The changes are reverted upon exit if executed within the model as context.

    Parameters
    ----------
    model: cobra.Model
        The model whose reaction bounds are to be set.
    gene_list: list of cobra.Gene
        The list of genes to knock-out.

SyntaxError: EOF while scanning triple-quoted string literal (883206811.py, line 17)

double = double_gene_deletion(nr_model)

print(double.knockout[{"b0480", "b4390"}])

b0480 = nr_model.gene.b0480
b4390 = nr_model.gene.b4390

print(double.knockout[{b0480, b4390}])